<a href="https://colab.research.google.com/github/avishank-dwivedi/Pytorch-Framework/blob/main/12cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, Dataset
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

In [ ]:
torch.manual_seed(42)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"using device: {device}")

In [ ]:
from google.colab import drive


# Mount Drive
drive.mount('/content/drive')

# Read the file (just adjust the path)
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/pytorch/fashion-mnist_train.csv')
df.head()

In [ ]:
df.shape

In [ ]:
#create a 4*4 grid of images
fig, axes = plt.subplots(4 , 4 , figsize=(10 , 10))
fig.suptitle("first 16 /images" , fontsize=16)

#plot the first 16 image from the dataset
for i , ax in enumerate(axes.flat):
    img = df.iloc[i , 1:].values.reshape(28 , 28) #Reshape to 28*28
    ax.imshow(img)# display in grayscale
    ax.axis('off') # Remove axis for a clear look
    ax.set_title(f"Label: {df.iloc[i , 0]}") #show the label

plt.tight_layout(rect = [0,0,1,0.96])
plt.show()

In [ ]:
# train test split
df = df.dropna()
X = df.iloc[:, 1:].values
y = df.iloc[:, 0].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
#Transformations
from torchvision.transforms import transforms

custom_transform = transforms.Compose([
transforms.Resize((256)),
transforms.CenterCrop(224),
transforms.ToTensor(),
transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [ ]:
from PIL import Image
import numpy as np

class CustomDataset(Dataset):

  def __init__(self , features , labels , transforms):
    self.features = features
    self.labels = labels
    self.transforms = transforms

  def __len__(self):
    return len(self.features)

  def __getitem__(self  , index):
    #resize to (28 , 28)
    image = self.features[index].reshape(28,28) # Corrected from self.feature

    #change dataset to np.unit8
    image = image.astype(np.uint8)

    #change black&white to colr - > (H,w , c)-> (c, h , w)
    image = np.stack([image]*3, axis=-1) # Added axis=0 for correct stacking

    #convert array to PIL image
    image = Image.fromarray(image) # This line is not needed as ToTensor will handle the conversion from numpy array

    #apply transforms
    image = self.transforms(image) # Apply transforms to a torch tensor

    #return
    return image , torch.tensor(self.labels[index] , dtype=torch.long)

In [ ]:
#create train dataset object
train_dataset = CustomDataset(X_train , y_train , transforms=custom_transform)

In [ ]:
len(train_dataset)

In [ ]:
#create test_dataset  object
test_dataset = CustomDataset(X_test , y_test , transforms=custom_transform)

In [ ]:
len(test_dataset)

In [ ]:
#create train and test loader
train_loader = DataLoader(train_dataset , batch_size=32 , shuffle=True)
test_loader = DataLoader(test_dataset , batch_size=32, shuffle=False)


In [ ]:
#featch the pretrain model

import torchvision.models as models

vgg16 = models.vgg16(pretrained=True)

In [ ]:
vgg16


In [ ]:
for parm in vgg16.features.parameters():
  parm.requires_grad = False

In [ ]:
vgg16.classifier = nn.Sequential(
    nn.Linear(25088 , 1024),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(1024 , 512),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(512 , 10)
)


In [ ]:
vgg16 = vgg16.to(device)

In [ ]:
learning_rate = 0.0001
num_epochs = 10

In [ ]:

#loss function
criterion = nn.CrossEntropyLoss()

#optimizer
optimizer = optim.Adam(vgg16.classifier.parameters() , lr=learning_rate)



In [ ]:
#training loop


for epoch in range(num_epochs):
  total_epoch_loss = 0
  for batch_features , batch_labels in train_loader:

    #move data on gpu

    batch_features , batch_labels = batch_features.to(device),batch_labels.to(device)

    #forward pass
    output = vgg16(batch_features)

    #calculate loss
    loss = criterion(output , batch_labels)

    #backward pass
    optimizer.zero_grad()
    loss.backward()

    #update weights
    optimizer.step()

    total_epoch_loss = total_epoch_loss + loss.item()
  avg_loss = total_epoch_loss/len(train_loader)
  print(f"Epoch {epoch+1}/{num_epochs} , Loss : {avg_loss}")

Epoch 1/10 , Loss : 0.36688610225170853
Epoch 2/10 , Loss : 0.21619102658952277
Epoch 3/10 , Loss : 0.16829753732153524


In [ ]:
#set model to eval mode

model.eval()

In [ ]:
#evaluation code

total =0
correct = 0

with torch.no_grad():
  for batch_features , batch_labels in test_loader:

     #move data on gpu

    batch_features , batch_labels = batch_features.to(device),batch_labels.to(device)


    output = vgg16(batch_features)

    _,predicted = torch.max(output.data , 1)

    total += batch_labels.shape[0]
    correct += (predicted == batch_labels).sum().item()
print(correct/total)